In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
fraud_df = pd.read_csv('fraud_dataset1.csv')

In [4]:
fraud_df.head()

,category,amt,trans_hour,distance_km,distance_from_last_transaction,amt_deviation,time_since_last_transaction,velocity_1hr,is_night,is_weekend,...,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,misc_pos,78.52,16,91.864216,84.495812,12.020516,199.900000,1.0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,gas_transport,65.25,7,121.877934,210.284759,1.249484,932.950000,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,kids_pets,87.74,15,65.393092,58.169994,21.240516,453.966667,1.0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,personal_care,148.02,12,38.394780,75.021641,81.520516,1256.383333,1.0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,shopping_net,2.83,14,62.145982,97.063040,63.669484,115.900000,1.0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
# checking for class 

fraud_df['is_fraud'].value_counts()

is_fraud
0    552670
1      2125
Name: count, dtype: int64

In [7]:
#extracting the 1s class and taking 2200 columns from 0s class

zero_df = fraud_df[fraud_df['is_fraud'] == 0]
one_df = fraud_df[fraud_df['is_fraud'] == 1]

In [8]:
zero_df_2200_cols = zero_df[:2200]

In [9]:
zero_df_2200_cols

,category,amt,trans_hour,distance_km,distance_from_last_transaction,amt_deviation,time_since_last_transaction,velocity_1hr,is_night,is_weekend,...,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,misc_pos,78.52,16,91.864216,84.495812,12.020516,199.900000,1.0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,gas_transport,65.25,7,121.877934,210.284759,1.249484,932.950000,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,kids_pets,87.74,15,65.393092,58.169994,21.240516,453.966667,1.0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,personal_care,148.02,12,38.394780,75.021641,81.520516,1256.383333,1.0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,shopping_net,2.83,14,62.145982,97.063040,63.669484,115.900000,1.0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,gas_transport,49.90,1,32.413445,49.065971,13.789516,500.950000,1.0,1,1,...,0,0,0,0,0,0,0,0,0,0
2204,shopping_pos,104.20,0,80.442495,83.957463,40.510484,1387.366667,1.0,1,0,...,0,0,0,0,0,0,0,0,1,0
2205,grocery_pos,124.24,6,110.926710,182.415484,60.550484,339.500000,1.0,0,0,...,1,0,0,0,0,0,0,0,0,0
2206,grocery_pos,120.98,8,129.406334,48.659323,57.290484,118.950000,4.0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
final_fraud_df = pd.concat([one_df, zero_df_2200_cols], axis=0)

In [12]:
final_fraud_df.head()

,category,amt,trans_hour,distance_km,distance_from_last_transaction,amt_deviation,time_since_last_transaction,velocity_1hr,is_night,is_weekend,...,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
1901,grocery_pos,337.39,1,117.193549,85.597483,245.544268,144.566667,1.0,1,0,...,1,0,0,0,0,0,0,0,0,0
1902,grocery_pos,292.54,2,115.903699,232.975679,200.694268,50.916667,1.0,1,0,...,1,0,0,0,0,0,0,0,0,0
1903,shopping_net,995.97,22,93.119251,55.482320,904.124268,1236.533333,1.0,0,0,...,0,0,0,0,0,0,0,1,0,0
1904,shopping_net,1062.66,23,106.067481,199.004622,970.814268,30.833333,1.0,0,0,...,0,0,0,0,0,0,0,1,0,0
1905,shopping_net,1113.08,23,73.585472,177.684018,1021.234268,37.700000,2.0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
final_fraud_df['is_fraud'].value_counts()

is_fraud
0    2200
1    2125
Name: count, dtype: int64

In [14]:
X_fraud = final_fraud_df.drop(['category', 'is_fraud'], axis=1)
y_fraud = final_fraud_df['is_fraud']

In [15]:
#Data normalization

scaler = StandardScaler()

X_fraud_scaled = scaler.fit_transform(X_fraud)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_fraud_scaled, y_fraud, test_size=0.2, random_state=42)

In [17]:
#model 1 ===> Logistic Regression

model_logistic = LogisticRegression()

model_logistic.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [18]:
model_logistic.score(X_train, y_train)

0.8794797687861272

In [19]:
model_logistic.score(X_test, y_test)

0.8809248554913295

In [20]:
#model 2 ===> XGBoost

model_xgb = XGBClassifier()

model_xgb.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [21]:
model_xgb.score(X_train, y_train)

1.0

In [23]:
model_xgb.score(X_test, y_test)

0.9699421965317919

In [24]:
#model 3 ===> Random Forest Classifier

model_random = RandomForestClassifier()

model_random.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [25]:
model_random.score(X_train, y_train)

1.0

In [26]:
model_random.score(X_test, y_test)

0.9641618497109826

In [ ]:
#Ok very good model, it will go with XGBoost Classifier because accuracy score is slightly higher then Random Forest Classifier